<a href="https://colab.research.google.com/github/jyothishri184/Market-basket-Analysis/blob/main/Pre_Processing_Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MARKET BASKET ANAYSIS


#DATA LOADING

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=05f04089a37f322d3d442a6673fae099e6826d494017de80116db941cb78bc7d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MarketBasketAnalysis").getOrCreate()

In [ ]:
df_1 = spark.read.csv("/online_retail_II_10_11.csv",header=True,inferSchema=True)

In [ ]:
df_1.show()

+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
|Invoice|StockCode|         Description|Quantity|     InvoiceDate|Price|Customer ID|       Country|
+-------+---------+--------------------+--------+----------------+-----+-----------+--------------+
| 536365|   85123A|WHITE HANGING HEA...|       6|01-12-2010 08:26| 2.55|      17850|United Kingdom|
| 536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
| 536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 08:26| 2.75|      17850|United Kingdom|
| 536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
| 536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 08:26| 3.39|      17850|United Kingdom|
| 536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 08:26| 7.65|      17850|United Kingdom|
| 536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 08:26| 4.25|      17850|United Kingdom|


#PREPROCESSING  


DATA EXPLORATION

In [ ]:
df_1.describe().show()

+-------+------------------+------------------+--------------------+------------------+----------------+-----------------+------------------+-----------+
|summary|           Invoice|         StockCode|         Description|          Quantity|     InvoiceDate|            Price|       Customer ID|    Country|
+-------+------------------+------------------+--------------------+------------------+----------------+-----------------+------------------+-----------+
|  count|            541910|            541910|              540456|            541910|          541910|           541910|            406830|     541910|
|   mean| 559965.7926209917|27623.240210938104|             20713.0| 9.552233765754462|            NULL|4.611138332927965|15287.684160460143|       NULL|
| stddev|13428.437355340984| 16799.73762842774|                NULL|218.08095694392543|            NULL|96.75976549366548|1713.6030743514768|       NULL|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|        

In [ ]:
df_1.dtypes

[('Invoice', 'string'),
 ('StockCode', 'string'),
 ('Description', 'string'),
 ('Quantity', 'int'),
 ('InvoiceDate', 'string'),
 ('Price', 'double'),
 ('Customer ID', 'int'),
 ('Country', 'string')]

**Attribute Information:**

**InvoiceNo:** Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.


**StockCode:** Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.


**Description:** Product (item) name. Nominal.


 **Quantity:** The quantities of each product (item) per transaction. Numeric.


 **InvoiceDate:**  Invice date and time. Numeric. The day and time when a transaction was generated.


 **UnitPrice:**  Unit price. Numeric. Product price per unit in sterling (Â£).


 **CustomerID:** Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.


 **Country:**  Country name. Nominal. The name of the country where a customer resides.

COLUMN RENAME

In [ ]:
df_1.columns

['Invoice',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'Price',
 'Customer ID',
 'Country']

In [ ]:
df_1 = df_1.withColumnRenamed("Invoice","transaction_id")
df_1 = df_1.withColumnRenamed("StockCode","product_id")
df_1 = df_1.withColumnRenamed("Description","product_name")
df_1 = df_1.withColumnRenamed("Quantity","quantity")
df_1 = df_1.withColumnRenamed("InvoiceDate","transaction_date")
df_1 = df_1.withColumnRenamed("Price","price")
df_1 = df_1.withColumnRenamed("Customer ID","customer_id")
df_1 = df_1.withColumnRenamed("Country","country")

In [ ]:
df_1.columns

['transaction_id',
 'product_id',
 'product_name',
 'quantity',
 'transaction_date',
 'price',
 'customer_id',
 'country']

DEALING WITH NULL VALUES

In [ ]:
print((df_1.count(), len(df_1.columns)))

(541910, 8)


In [ ]:
df_1 = df_1.dropna()

In [ ]:
print((df_1.count(), len(df_1.columns)))

(406830, 8)


REMOVING DUPLICATES


In [ ]:
print((df_1.count(), len(df_1.columns)))

(406830, 8)


In [ ]:
df_1 = df_1.dropDuplicates()

In [ ]:
print((df_1.count(), len(df_1.columns)))

(401605, 8)


IN Column 'Transation id' removing rows staring with 'C' SINCE they are cancelled order

In [ ]:
from pyspark.sql.functions import col

In [ ]:
df_1 = df_1.filter(~col("transaction_id").startswith("C"))
df_1.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536381|     22411|JUMBO SHOPPER VIN...|      10|01-12-2010 09:41| 1.95|      15311|United Kingdom|
|        536392|     22827|RUSTIC  SEVENTEEN...|       1|01-12-2010 10:29|165.0|      13705|United Kingdom|
|        536401|     21592|RETROSPOT CIGAR B...|       1|01-12-2010 11:21| 1.25|      15862|United Kingdom|
|        536464|     22988|   SOLDIERS EGG CUP |       1|01-12-2010 12:23| 1.25|      17968|United Kingdom|
|        536464|     22910|PAPER CHAIN KIT V...|       1|01-12-2010 12:23| 2.95|      17968|United Kingdom|
|        536488|     20878|SET/9 CHRISTMAS T...|       2|01-12-2010 12:31| 1.25|      17897|United Kingdom|
|        536500|     22712| 

In [ ]:
print((df_1.count(), len(df_1.columns)))

(392733, 8)


converting column 'TRANSACTION_ID' to INT

In [ ]:
df_1 = df_1.withColumn("transaction_id", df_1["transaction_id"].cast("int"))
df_1.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536381|     22411|JUMBO SHOPPER VIN...|      10|01-12-2010 09:41| 1.95|      15311|United Kingdom|
|        536392|     22827|RUSTIC  SEVENTEEN...|       1|01-12-2010 10:29|165.0|      13705|United Kingdom|
|        536401|     21592|RETROSPOT CIGAR B...|       1|01-12-2010 11:21| 1.25|      15862|United Kingdom|
|        536464|     22988|   SOLDIERS EGG CUP |       1|01-12-2010 12:23| 1.25|      17968|United Kingdom|
|        536464|     22910|PAPER CHAIN KIT V...|       1|01-12-2010 12:23| 2.95|      17968|United Kingdom|
|        536488|     20878|SET/9 CHRISTMAS T...|       2|01-12-2010 12:31| 1.25|      17897|United Kingdom|
|        536500|     22712| 

In [ ]:
df_1.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'string'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

Spliting 'transaction_date' column to 'transaction_date' and 'transaction_time'

In [ ]:
df_1 = df_1

In [ ]:
df_1.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536381|     22411|JUMBO SHOPPER VIN...|      10|01-12-2010 09:41| 1.95|      15311|United Kingdom|
|        536392|     22827|RUSTIC  SEVENTEEN...|       1|01-12-2010 10:29|165.0|      13705|United Kingdom|
|        536401|     21592|RETROSPOT CIGAR B...|       1|01-12-2010 11:21| 1.25|      15862|United Kingdom|
|        536464|     22988|   SOLDIERS EGG CUP |       1|01-12-2010 12:23| 1.25|      17968|United Kingdom|
|        536464|     22910|PAPER CHAIN KIT V...|       1|01-12-2010 12:23| 2.95|      17968|United Kingdom|
|        536488|     20878|SET/9 CHRISTMAS T...|       2|01-12-2010 12:31| 1.25|      17897|United Kingdom|
|        536500|     22712| 

In [ ]:
df_1.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'string'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

In [ ]:
from pyspark.sql.functions import substring,to_date

df_1 = df_1.withColumn('transaction_date', to_date(substring(df_1['transaction_date'], 1, 10), 'dd-MM-yyyy'))

df_1.show()

+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|transaction_id|product_id|        product_name|quantity|transaction_date|price|customer_id|       country|
+--------------+----------+--------------------+--------+----------------+-----+-----------+--------------+
|        536381|     22411|JUMBO SHOPPER VIN...|      10|      2010-12-01| 1.95|      15311|United Kingdom|
|        536392|     22827|RUSTIC  SEVENTEEN...|       1|      2010-12-01|165.0|      13705|United Kingdom|
|        536401|     21592|RETROSPOT CIGAR B...|       1|      2010-12-01| 1.25|      15862|United Kingdom|
|        536464|     22988|   SOLDIERS EGG CUP |       1|      2010-12-01| 1.25|      17968|United Kingdom|
|        536464|     22910|PAPER CHAIN KIT V...|       1|      2010-12-01| 2.95|      17968|United Kingdom|
|        536488|     20878|SET/9 CHRISTMAS T...|       2|      2010-12-01| 1.25|      17897|United Kingdom|
|        536500|     22712| 

In [ ]:
df_1.dtypes

[('transaction_id', 'int'),
 ('product_id', 'string'),
 ('product_name', 'string'),
 ('quantity', 'int'),
 ('transaction_date', 'date'),
 ('price', 'double'),
 ('customer_id', 'int'),
 ('country', 'string')]

In [ ]:
import os
pandas_df = df_1.toPandas()

local_directory = r"C:\Users\jyoth\Downloads\Projects\MarketBasketAnalysis"

os.makedirs(local_directory, exist_ok=True)

file_name = "local_output.csv"

local_file_path = os.path.join(local_directory, file_name)

pandas_df.to_csv(local_file_path, index=False)

In [ ]:
from google.colab import files

files.download("local_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>